In [ ]:
%matplotlib inline
# do the imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import layers
from keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# set up the id block as a function
def idBlock(x, filters, kernel_size):
    # for the id block the short cut is just the input
    shortcut = x

    # since this is a resnet 18 we have two main blocks or sections
    # we have a convoliutonal layer => batchnorm with a relu activation funding
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    # here we add the shortcut with the output of the layers => this is like the skip connection!
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x

# this is the convolutional block
# this has the same connection skipping that the id block has but it changes the dimensions
def convBlock(x, filters, kernel_size, stride):

    # first lets save the shortcut like before so we can use it at the end to add them
    # to get the skip connection
    shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same', kernel_initializer='he_normal')(x)
    shortcut = layers.BatchNormalization()(shortcut)

    # now we have that first convolutional layer
    # for this one we keep the same stide as the shortcut => this is to maintain the same space
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # like the id block since this is a resnet-18 we have to convo sets => same as above essentially
    x = layers.Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)

    # now we can add to get that skip connection
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)

    return x


https://medium.com/analytics-vidhya/understanding-and-implementation-of-residual-networks-resnets-b80f9a507b9c


In [ ]:
def create_RESNET_18(input_shape, num_classes):
    # get the inputs
    inputs = layers.Input(shape=input_shape)

    # this is the 1/5 stage
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # 2/5 stage
    x = convBlock(x, filters=64, kernel_size=3, stride=1)
    x = idBlock(x, filters=64, kernel_size=3)

    # 3/5 stage
    x = convBlock(x, filters=128, kernel_size=3, stride=2)
    x = idBlock(x, filters=128, kernel_size=3)

    # 4/5 stage
    x = convBlock(x, filters=256, kernel_size=3, stride=2)
    x = idBlock(x, filters=256, kernel_size=3)

    # 5/5 stage
    x = convBlock(x, filters=512, kernel_size=3, stride=2)
    x = idBlock(x, filters=512, kernel_size=3)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(x)

    model = tf.keras.Model(inputs=inputs, outputs=x, name='ResNet')

    model.compile(
      optimizer = 'adam',
      loss = "sparse_categorical_crossentropy",
      metrics = ['accuracy']
    )

    return model

In [ ]:
# read in the data
# main directory is from my google drive
# load the images with TensorFlow => labels are inferred from directory strcture
def readData():
  main_directory = '/content/drive/My Drive/week_1_data'

  dataset = tf.keras.utils.image_dataset_from_directory(
      main_directory,
      labels='inferred',
      label_mode='int',
      image_size=(128, 128),
      batch_size=32,
      shuffle=True
  )

  # iterate through the images/labels for each batch
  # append the data/labels to each list so we can concatenate them into np arrays
  images = []
  labels = []
  for image_batch, label_batch in dataset:
      images.append(image_batch.numpy())
      labels.append(label_batch.numpy())
  return np.concatenate(images), np.concatenate(labels)

images, labels = readData()


Found 485 files belonging to 3 classes.


In [ ]:
# Define input shape based on your image data
input_shape = (128, 128, 3)
num_classes = 3


x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
model = create_RESNET_18(input_shape, num_classes)
model.fit(x_train, y_train, epochs=14, batch_size=16)
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy}')

Epoch 1/14
25/25 [==============================] - 16s 39ms/step - loss: 1.4543 - accuracy: 0.5387
Epoch 2/14
25/25 [==============================] - 1s 33ms/step - loss: 0.8234 - accuracy: 0.6701
Epoch 3/14
25/25 [==============================] - 1s 35ms/step - loss: 0.8480 - accuracy: 0.6521
Epoch 4/14
25/25 [==============================] - 1s 38ms/step - loss: 0.6805 - accuracy: 0.6856
Epoch 5/14
25/25 [==============================] - 1s 32ms/step - loss: 0.5574 - accuracy: 0.7680
Epoch 6/14
25/25 [==============================] - 1s 32ms/step - loss: 0.4917 - accuracy: 0.7964
Epoch 7/14
25/25 [==============================] - 1s 32ms/step - loss: 0.3428 - accuracy: 0.8660
Epoch 8/14
25/25 [==============================] - 1s 34ms/step - loss: 0.3473 - accuracy: 0.8763
Epoch 9/14
25/25 [==============================] - 1s 42ms/step - loss: 0.4657 - accuracy: 0.8170
Epoch 10/14
25/25 [==============================] - 1s 35ms/step - loss: 0.4101 - accuracy: 0.8479
Epoch 11

Epoch 1/15
25/25 [==============================] - 12s 34ms/step - loss: 1.6721 - accuracy: 0.5180
Epoch 2/15
25/25 [==============================] - 1s 31ms/step - loss: 0.9693 - accuracy: 0.5979
Epoch 3/15
25/25 [==============================] - 1s 31ms/step - loss: 0.7303 - accuracy: 0.6649
Epoch 4/15
25/25 [==============================] - 1s 34ms/step - loss: 0.5590 - accuracy: 0.7526
Epoch 5/15
25/25 [==============================] - 1s 34ms/step - loss: 0.4666 - accuracy: 0.8119
Epoch 6/15
25/25 [==============================] - 1s 34ms/step - loss: 0.4440 - accuracy: 0.8273
Epoch 7/15
25/25 [==============================] - 1s 35ms/step - loss: 0.3770 - accuracy: 0.8557
Epoch 8/15
25/25 [==============================] - 1s 31ms/step - loss: 0.2936 - accuracy: 0.8866
Epoch 9/15
25/25 [==============================] - 1s 31ms/step - loss: 0.2487 - accuracy: 0.9201
Epoch 10/15
25/25 [==============================] - 1s 31ms/step - loss: 0.1591 - accuracy: 0.9278
Epoch 11/15
25/25 [==============================] - 1s 31ms/step - loss: 0.3013 - accuracy: 0.8840
Epoch 12/15
25/25 [==============================] - 1s 31ms/step - loss: 0.3406 - accuracy: 0.8892
Epoch 13/15
25/25 [==============================] - 1s 31ms/step - loss: 0.2950 - accuracy: 0.9072
Epoch 14/15
25/25 [==============================] - 1s 31ms/step - loss: 0.1865 - accuracy: 0.9330
Epoch 15/15
25/25 [==============================] - 1s 31ms/step - loss: 0.1381 - accuracy: 0.9459
4/4 [==============================] - 1s 20ms/step - loss: 1.0111 - accuracy: 0.7629
Test accuracy: 0.7628865838050842
